<a target="_blank" href="https://colab.research.google.com/github/Tensor-Reloaded/Neural-Networks-Template-2024/blob/main/Lab01/Assignment1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Assignment 1 (10 points)**

## **Solving a linear system in python**

In this homework, you will familiarize yourself with key linear algebra con-
cepts and Python programming by solving a system of linear equations. You
will explore multiple methods for solving such systems, including Cramer’s rule
and matrix inversion. By the end of this assignment, you will have a good un-
derstanding of how to represent and manipulate matrices and vectors in Python.

We begin with the following system of 3 linear equations with 3 unknowns:
$$ 2x + 3y - z = 5 $$
$$ x - y + 4z = 6 $$
$$ 3x + y + 2z = 7 $$

This system can be vectorized in the following form:
$$ A \cdot X = B $$
where:
$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -1 & 4 \\
3 & 1 & 2
\end{bmatrix}, \quad
X = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}, \quad
B = \begin{bmatrix}
5 \\
6 \\
7
\end{bmatrix}
$$

**Considerations**
- do not use any linear algebra framework such as $numpy$
- use python lists as data structures for matrices and vectors
- experiment with other values for the coefficients and free terms

### **1. Parsing the System of Equations (1 point)**

The first task is to implement a Python script that reads a system of linear equations from a text file and parses it into a matrix $A$ and a vector $B$. You will use the input format described below to extract the coefficients for $A$ and $B$.

**Input File Format**
```text
2x + 3y - z = 5
x - y + 4z = 6
3x + y + 2z = 7
```

Note that the coefficients are always in the order x, y and z and the terms are always space separated

In [14]:
import pathlib
#from google.colab import files
#uploaded = files.upload()


def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
  A = []
  B = []
  with path.open('r') as file:
    for line in file:
      left_side, right_side = line.split('=')
      left_side = left_side.replace(' ','')
      a = []
      sign = 1
      coef_chr = ''
      for chr in left_side:
        if chr.isdigit() or chr == '.':
          coef_chr = coef_chr + chr
        elif chr == '+':
          sign = 1
        elif chr == '-':
          sign = -1
        else:
          if coef_chr == '':
            coef_chr = '1'
          coef = float(coef_chr)
          if sign == -1:
            coef = -coef
          a.append(coef)
          coef_chr = ''

      A.append(a)

      B.append(float(right_side))
    return A, B

A, B = load_system(pathlib.Path("system.txt"))
print(f"{A=} {B=}")

A=[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]] B=[5.0, 6.0, 7.0]


### **2. Matrix and Vector Operations (5 points)**

Once you have successfully parsed the matrix and vector, complete the following exercises to manipulate and understand basic matrix and vector operations. Write Python functions for each of these tasks:

#### 2.1. Determinant

Write a function to compute the determinant of matrix $A$. Recall one of the formulae for the determinant of a $3x3$ matrix:
$$ \text{det}(A) = a_{11}(a_{22}a_{33} - a_{23}a_{32}) - a_{12}(a_{21}a_{33} - a_{23}a_{31}) + a_{13}(a_{21}a_{32} - a_{22}a_{31}) $$

In [4]:
def determinant(matrix: list[list[float]]) -> float:
  a11, a12, a13 = matrix[0]
  a21, a22, a23 = matrix[1]
  a31, a32, a33 = matrix[2]

  det = (a11*(a22*a33 - a23*a32)
          - a12*(a21*a33 - a23*a31)
          + a13*(a21*a32 - a22*a31))

  return det

print(f"{determinant(A)=}")

determinant(A)=14.0


#### 2.2. Trace

Compute the sum of the elements along the main diagonal of matrix $A$. For a matrix $A$, this is:
$$ \text{Trace}(A) = a_{11} + a_{22} + a_{33} $$

In [6]:
def trace(matrix: list[list[float]]) -> float:
  result = 0
  for i in range(len(matrix)):
    result += matrix[i][i]
  return result

print(f"{trace(A)=}")

trace(A)=3.0


#### 2.3. Vector norm

Compute the Euclidean norm of vector $B$, which is:
$$ ||B|| = \sqrt{b_1^2 + b_2^2 + b_3^2} $$

In [7]:
def norm(vector: list[float]) -> float:
  result = 0
  for i in range(len(vector)):
    result += vector[i]**2

  result = result**0.5
  return result

print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix

Write a function to compute the transpose of matrix $A$. The transpose of a matrix $A$ is obtained by swapping its rows and columns.
    

In [8]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
  transposed_matrix = []

  for _ in range(len(matrix[0])):
    transposed_matrix.append([0.0]*len(matrix))

  for i in range(len(matrix)):
    for j in range(len(matrix[0])):
      transposed_matrix[j][i] = matrix[i][j]

  return transposed_matrix
  #sau
  #return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]

print(f"{transpose(A)=}")

transpose(A)=[[2.0, 1.0, 3.0], [3.0, -1.0, 1.0], [-1.0, 4.0, 2.0]]


#### 2.5. Matrix-vector multiplication

Write a function that multiplies matrix $A$ with vector $B$.

In [9]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
  result = [0.0]*len(matrix)

  for i in range(len(matrix)):
    for j in range(len(vector)):
      result[i] += matrix[i][j]*vector[j]

  return result

print(f"{multiply(A, B)=}")

multiply(A, B)=[21.0, 27.0, 35.0]


### **3. Solving using Cramer's Rule (1 point)**

Now that you have explored basic matrix operations, solve the system of linear equations using Cramer's rule.

**Cramer's Rule:**

Cramer's rule allows you to solve for each unknown $x$, $y$, and $z$ using determinants. For example:
$$ x = \frac{\text{det}(A_x)}{\text{det}(A)}, \quad y = \frac{\text{det}(A_y)}{\text{det}(A)}, \quad z = \frac{\text{det}(A_z)}{\text{det}(A)} $$
where $A_x$, $A_y$, and $A_z$ are matrices formed by replacing the respective column of matrix $A$ with vector $B$.

In [10]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
  solution = []
  det = determinant(matrix)
  for param in range(len(matrix)):
    matrix_param = [row[:] for row in matrix] #deep copy

    for j in range(len(vector)):
      matrix_param[j][param] = vector[j]

    param_value = determinant(matrix_param)/det
    solution.append(param_value)

  return solution

print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


### **4. Solving using Inversion (3 points)**

Finally, solve the system by computing the inverse of matrix $A$ and multiplying it by vector $B$.
$$ A \cdot X = B \rightarrow X = A^{-1} \cdot B $$
**Adjugate Method for Matrix Inversion:**

To find the inverse of matrix $ A $, you can use the adjugate method:
$$ A^{-1} = \frac{1}{\text{det}(A)} \times \text{adj}(A) $$
where $\text{adj}(A)$ is the adjugate (or adjoint) matrix, which is the transpose of the cofactor matrix of $ A $.

**Cofactor Matrix:**

The cofactor matrix is a matrix where each element is replaced by its cofactor. The cofactor of an element $a_{ij}$ is given by:
$$ (-1)^{i+j} \times \text{det}(M_{ij}) $$
where $M_{ij}$ is the minor of element $a_{ij}$, which is the matrix obtained by removing the $i$-th row and $j$-th column from matrix $A$.

In [11]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
  minor_matrix = []

  for row in range(len(matrix)):
    if row == i:
      continue

    minor_row = []
    for col in range(len(matrix[0])):
      if col == j:
        continue
      minor_row.append(matrix[row][col])

    minor_matrix.append(minor_row)

  return minor_matrix

def determinant_laplace(matrix: list[list[float]]) -> float:
  if len(matrix) == 1:
    return matrix[0][0]

  if len(matrix) == 2:
    return matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0]

  det = 0.0
  for j in range(len(matrix)):
    minor_matrix = minor(matrix, 0, j)
    det += ((-1)**j) * matrix[0][j] * determinant_laplace(minor_matrix)

  return det


def cofactor(matrix: list[list[float]]) -> list[list[float]]:
  cofactor_matrix = []
  for i in range(len(matrix)):
    cofactor_row = []
    for j in range(len(matrix[0])):
      cofactor_row.append(((-1)**(i + j)) * determinant_laplace(minor(matrix, i, j)))

    cofactor_matrix.append(cofactor_row)

  return cofactor_matrix

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
  adjoint_matrix = transpose(cofactor(matrix))
  return adjoint_matrix

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
  solution = []
  det = determinant(matrix)
  inverse_matrix = adjoint(matrix)

  for i in range(len(matrix)):
    for j in range(len(matrix[0])):
      inverse_matrix[i][j] /= det

  solution = multiply(inverse_matrix, vector)
  return solution


print(f"{solve(A, B)=}")

solve(A, B)=[0.35714285714285765, 2.071428571428571, 1.9285714285714288]


**Bonus**: Can you see any similarities between the cofactor and the provided
formula for the determinant of the matrix A?

Yes, ilustrated by the Laplace formula. In particular, for the 3x3 matrix we have:

det(A) = a11(a22a33−a23a32)−a12(a21a33−a23a31)+a13(a21a32−a22a31) = a11det(M11) - a12det(M12) + a13det(M13) = (-1)^(1+1)a11det(M11) + (-1)^(1+2)a12det(M12) + (-1)^(1+3)a13det(M13)


Function for determinant using Laplace formula:


```
def determinant_laplace(matrix: list[list[float]]) -> float:
  if len(matrix) == 1:
    return matrix[0][0]

  if len(matrix) == 2:
    return matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0]

  det = 0.0
  for j in range(len(matrix)):
    minor_matrix = minor(matrix, 0, j)
    det += ((-1)**j) * matrix[0][j] * determinant_laplace(minor_matrix)

  return det
```



